# Manual Discord Data Export 

Since Discord doesn't have a public API, and using other scripts will violate the Discord terms and conditions, we will need to manually export the data.

**Open Discord** -> **Privacy & Safety** -> **Request Data Export** 

**Important**: Uncheck everything except messages. This will force script to work properly.

In [12]:
import json 
import os
import numpy as np
import pandas as pd

In [38]:
# Path to the directory where you extracted the Discord data
path = "package"

# Function to extract information from messages.json
def extract_message_info(json_file_path, message_limit: int = None):
    with open(json_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        for row in data:
            timestamp = row.get('Timestamp')
            content = row.get('Contents')
            return [content, timestamp]

In [82]:
def main(message_limit: int = None, dialogs_limit: int = None, verbose=1, threshold: int = 50):
      if not os.path.exists(path):
            print(f"Discord Directory '{path}' does not exist.")
            exit()

      data = []
      limit = False
      if verbose: 
            print(f"Discord data is processed from Path: {path}")

      # Walks through every dialog in the discord dialogs directory
      for root, dirs, files in os.walk(path):
            if not limit:
                  for file in files[:dialogs_limit]:
                        if file == 'messages.json':
                              json_file_path = os.path.join(root, file)
                              processed_row = extract_message_info(json_file_path, message_limit)
                              data.append(processed_row)
                              
                              # if limit
                              if message_limit and len(data) >= message_limit:
                                    limit= True
                                    break
      
      try: 
            data = pd.DataFrame(data, columns=['Message', 'Date'])
            return data
      except Exception as e:
            print(f"Fixing exception: {e}") # Clears all None values
            data = [row for row in data if row is not None]
            data = pd.DataFrame(data, columns=['Message', 'Date'])
            return data

In [ ]:
data = main(dialogs_limit=10) # Len of data will be less than limit because of how pandas df transforming.